In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -q -r '/content/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

### Import the required libraries

In [ ]:
import os
import sys
import torch
import numpy as np
import logging

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from collections import Counter
import random

logging.getLogger().setLevel(logging.INFO)

### Load model and tokenizer

In [ ]:
MODEL_NAME = "distilbert-base-uncased"
DESIRED=5

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')


device = get_device()
model.to(device)

print(f"Using device: {device}")

Using device: cuda


### Prepare data for training

In [ ]:
TRAIN_DATASET_PATH = os.path.join("data", "train.csv")
VAL_DATASET_PATH = os.path.join("data", "val.csv")

train_dataset = load_dataset('csv', data_files=TRAIN_DATASET_PATH)
val_dataset = load_dataset('csv', data_files=VAL_DATASET_PATH)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def undersample_dataset(dataset, label_counts, positive_label, positive_count):
    undersampled_indices = []
    # Collect indices for each original label
    label_indices = {label: [i for i, lbl in enumerate(dataset["original_labels"]) if lbl == label] for label in label_counts.keys()}

    # Sample the desired number of positive samples
    undersampled_indices.extend(random.sample(label_indices[positive_label], min(positive_count, len(label_indices[positive_label]))))

    # Sample the desired number of negative samples from each original label
    for label in label_counts:
        if label != positive_label:
            count_to_sample = min(positive_count // (len(label_counts) - 1), len(label_indices[label]))
            undersampled_indices.extend(random.sample(label_indices[label], count_to_sample))

    return dataset.select(undersampled_indices)

def prepare_dataset_for_bert_training(dataset):
    desired_label=DESIRED
    # Keep the original labels for undersampling
    dataset = dataset.rename_column("label", "original_labels")

    # Transform labels: 1 if label == desired_label else 0
    dataset = dataset.map(lambda examples: {"labels": [1 if label == desired_label else 0 for label in examples["original_labels"]]}, batched=True)

    # Tokenize the text data in the dataset
    dataset = dataset.map(lambda examples: tokenizer(examples["text"], truncation=True, padding=True), batched=True)

    # Count occurrences of each original label
    original_label_counts = Counter(dataset["original_labels"])

    # Calculate the total number of positive samples (label equal to desired_label)
    positive_count = original_label_counts[desired_label]

    # Update label counts for undersampling
    label_counts = {label: positive_count // (len(original_label_counts) - 1) for label in original_label_counts}
    label_counts[desired_label] = positive_count

    # Undersample the dataset
    undersampled_dataset = undersample_dataset(dataset, label_counts, desired_label, positive_count)

    return undersampled_dataset

In [ ]:
train_dataset_tokenized = prepare_dataset_for_bert_training(train_dataset["train"]).shuffle(42)
print(f"# Train dataset size: {len(train_dataset_tokenized)}")
val_dataset_tokenized = prepare_dataset_for_bert_training(val_dataset["train"])
print(f"# Validation dataset size: {len(val_dataset_tokenized)}")

Map:   0%|          | 0/20489 [00:00<?, ? examples/s]

Map:   0%|          | 0/20489 [00:00<?, ? examples/s]

# Train dataset size: 12291


Map:   0%|          | 0/871 [00:00<?, ? examples/s]

Map:   0%|          | 0/871 [00:00<?, ? examples/s]

# Validation dataset size: 722


### Prepare metrics

In [ ]:
f1 = load_metric("f1")
precision = load_metric("precision")
recall = load_metric("recall")

def computre_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "f1": f1.compute(predictions=predictions, references=labels, average='weighted')['f1'],
        "precision": precision.compute(predictions=predictions, references=labels, average='weighted')['precision'],
        "recall": recall.compute(predictions=predictions, references=labels, average='weighted')['recall']
    }

<ipython-input-9-4198f0db85cc>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1 = load_metric("f1")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


### Train model

In [ ]:
OUTPUT_DIR = os.path.join("models", f"distilbert_{DESIRED}")
os.environ["WANDB_PROJECT"] = f"distilbert_{DESIRED}"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=30,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=3,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    run_name=f"distilbert_{DESIRED}",
    report_to="wandb",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    compute_metrics=computre_metrics
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,F1,Precision,Recall
500,0.387100,0.296117,0.882632,0.889032,0.883657
1000,0.413500,0.321137,0.848859,0.879441,0.853186
1500,0.327800,0.250171,0.897527,0.897560,0.897507
2000,0.294000,0.280508,0.901652,0.901645,0.901662
2500,0.245600,0.246092,0.907931,0.913022,0.908587
3000,0.384100,0.246101,0.917586,0.924280,0.918283
3500,0.176600,0.292113,0.919284,0.922145,0.919668
4000,0.110700,0.261193,0.927694,0.929863,0.927978
4500,0.281400,0.207837,0.924709,0.929487,0.925208
5000,0.198200,0.284941,0.923624,0.924722,0.923823


Step,Training Loss,Validation Loss,F1,Precision,Recall
500,0.387100,0.296117,0.882632,0.889032,0.883657
1000,0.413500,0.321137,0.848859,0.879441,0.853186
1500,0.327800,0.250171,0.897527,0.897560,0.897507
2000,0.294000,0.280508,0.901652,0.901645,0.901662
2500,0.245600,0.246092,0.907931,0.913022,0.908587
3000,0.384100,0.246101,0.917586,0.924280,0.918283
3500,0.176600,0.292113,0.919284,0.922145,0.919668
4000,0.110700,0.261193,0.927694,0.929863,0.927978
4500,0.281400,0.207837,0.924709,0.929487,0.925208
5000,0.198200,0.284941,0.923624,0.924722,0.923823


TrainOutput(global_step=7685, training_loss=0.21978061894123324, metrics={'train_runtime': 3339.4382, 'train_samples_per_second': 18.403, 'train_steps_per_second': 2.301, 'total_flos': 8140783984404480.0, 'train_loss': 0.21978061894123324, 'epoch': 5.0})

### Save model

In [ ]:
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

('models/distilbert_5/tokenizer_config.json',
 'models/distilbert_5/special_tokens_map.json',
 'models/distilbert_5/vocab.txt',
 'models/distilbert_5/added_tokens.json',
 'models/distilbert_5/tokenizer.json')

### Push to hub

In [ ]:
MODEL_HUB_PATH = f"deptage/distilbert{DESIRED}"

model.push_to_hub(MODEL_HUB_PATH)
tokenizer.push_to_hub(MODEL_HUB_PATH)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/deptage/distilbert5/commit/faea70fcfddbc04e4f3e23c748ebbafd6ab72e4d', commit_message='Upload tokenizer', commit_description='', oid='faea70fcfddbc04e4f3e23c748ebbafd6ab72e4d', pr_url=None, pr_revision=None, pr_num=None)